<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL2.0/blob/main/06_content_recomm/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#06. Content Recommendations

**Writing a query to return page recommendations
to a social media user based on the
pages that their friends have liked, but
that they have not yet marked as liked.**

In [ ]:
!pip install polars
import pandas as pd
import numpy as  np
import polars as pl

In [ ]:
data1 = {'user_id': [1,1,1,2,3,3,4,4],
         'friend'  : [2,3,4,1,1,4,1,3]
        }

data2 = {'user_id' : [1,1,1,2,3,3,4],
         'page_likes' : ['A','B','C','A','B','C','B']
        }

pd_friends=pd.DataFrame(data1)
pd_likes=pd.DataFrame(data2)
pl_friends=pl.LazyFrame(data1)
pl_likes=pl.LazyFrame(data2)

In [ ]:
print(f'----PANDAS----\nfriends table:\n{pd_friends}')

----PANDAS----
friends table:
   user_id  friend
0        1       2
1        1       3
2        1       4
3        2       1
4        3       1
5        3       4
6        4       1
7        4       3


In [ ]:
print(f'likes table:\n{pd_likes}')

likes table:
   user_id page_likes
0        1          A
1        1          B
2        1          C
3        2          A
4        3          B
5        3          C
6        4          B


In [ ]:
pd_df1=(pd.merge(pd_friends
                 ,pd_likes.rename(columns={'user_id':'friend'
                                           ,'page_likes':'recommendation'
                                          }
                           )
                 ,on='friend')[['user_id','recommendation']]
)
print(f'Identifying possible recommendations:\n{pd_df1}')

Identifying possible recommendations:
    user_id recommendation
0         1              A
1         1              B
2         1              C
3         1              B
4         2              A
5         2              B
6         2              C
7         3              A
8         3              B
9         3              C
10        3              B
11        4              A
12        4              B
13        4              C
14        4              B
15        4              C


In [ ]:
pd_df=(pd.merge(pd_friends
                ,pd_likes.rename(columns={'user_id':'friend'}
                          )
                ,on='friend'
          )
)
pd_recommendation=(pd.merge(pd_likes,
                            pd_df,
                            on=['user_id','page_likes'],
                            how='outer',
                            indicator=True
                      )
                     .rename(columns={'page_likes':'recommendation'})
                     .query("_merge=='right_only'")[['user_id','recommendation']]
                     .drop_duplicates()
                     .sort_values(by='user_id')
)
print(f'Recommendations:')
pd_recommendation

Recommendations:


,user_id,recommendation
5,2,B
6,2,C
7,3,A
11,4,A
14,4,C


In [ ]:
quick1=pl_friends.head(5).collect()
print(f'----POLARS----\nfriends table sample:\n{quick1}')

----POLARS----
friends table sample:
shape: (5, 2)
┌─────────┬────────┐
│ user_id ┆ friend │
│ ---     ┆ ---    │
│ i64     ┆ i64    │
╞═════════╪════════╡
│ 1       ┆ 2      │
│ 1       ┆ 3      │
│ 1       ┆ 4      │
│ 2       ┆ 1      │
│ 3       ┆ 1      │
└─────────┴────────┘


In [ ]:
quick2=(pl_likes.head(5)
                .collect()
)
print(f'likes table sample:\n{quick2}')

likes table sample:
shape: (5, 2)
┌─────────┬────────────┐
│ user_id ┆ page_likes │
│ ---     ┆ ---        │
│ i64     ┆ str        │
╞═════════╪════════════╡
│ 1       ┆ A          │
│ 1       ┆ B          │
│ 1       ┆ C          │
│ 2       ┆ A          │
│ 3       ┆ B          │
└─────────┴────────────┘


In [ ]:
pl_lf1=(pl_friends.join(pl_likes.select(pl.col('user_id')
                                          .alias('friend'),
                                        pl.col('page_likes')
                                          .alias('recommendation')
                                 )
                        ,on='friend'
                        ,how='right'
                   )
).head(5).collect()
print(f'Identifying possible recommendations (sample):\n{pl_lf1}')

Identifying possible recommendations (sample):
shape: (5, 3)
┌─────────┬────────┬────────────────┐
│ user_id ┆ friend ┆ recommendation │
│ ---     ┆ ---    ┆ ---            │
│ i64     ┆ i64    ┆ str            │
╞═════════╪════════╪════════════════╡
│ 2       ┆ 1      ┆ A              │
│ 3       ┆ 1      ┆ A              │
│ 4       ┆ 1      ┆ A              │
│ 2       ┆ 1      ┆ B              │
│ 3       ┆ 1      ┆ B              │
└─────────┴────────┴────────────────┘


In [ ]:
pl_lf=(pl_friends.join(pl_likes.select(pl.col('user_id')
                                         .alias('friend'),
                                       pl.col('page_likes')
                                )
                       ,on='friend'
                       ,how='right'
                  )
                 .select(pl.col('user_id'),
                         pl.col('page_likes')
                  )
)
pl_recommendations=(pl_lf.join(pl_likes,
                               on=['user_id','page_likes'],
                               how='anti'
                          )
                         .unique()
                         .select(pl.col('user_id'),
                                 pl.col('page_likes')
                                   .alias('recommendation')
                          )
                         .sort(by='user_id')

).collect()
print(f'Recommendations:')
pl_recommendations

Recommendations:


user_id,recommendation
i64,str
2,"""B"""
2,"""C"""
3,"""A"""
4,"""C"""
4,"""A"""
